In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [2]:
book_folder = "C:/Praveen/deribit/book_snapshot_5/"
trade_folder = "C:/Praveen/deribit/trades/"

In [3]:
modified_book_folder = "C:/Praveen/Python/AlphaHFT/temp_books/"
modified_trade_folder = "C:/Praveen/Python/AlphaHFT/temp_trades/"

In [4]:
data_folder = "C:/Praveen/Python/AlphaHFT/"

In [5]:
books = [join(book_folder, f) for f in listdir(book_folder) if isfile(join(book_folder, f))]
data_dfs = []
for book in books:
    tokens = os.path.basename(book).split("_")
    date_holder = tokens[4]
    symbol = tokens[5]
    trade = r"deribit_trades_{0}_{1}".format(date_holder, symbol)
    trade = join(trade_folder, trade)
    #book_df = pd.read_csv(book, header=0, index_col='local_timestamp')
    #trade_df = pd.read_csv(trade, header=0, index_col='local_timestamp')
    data = join(data_folder, r"data_{0}_{1}".format(date_holder, symbol))
    print(data)
    data_df = pd.read_csv(data, header=0, index_col='timestamp')
    data_dfs.append(data_df)

C:/Praveen/Python/AlphaHFT/data_2024-01-01_BTC-PERPETUAL.csv.gz
C:/Praveen/Python/AlphaHFT/data_2024-02-01_BTC-PERPETUAL.csv.gz
C:/Praveen/Python/AlphaHFT/data_2024-03-01_BTC-PERPETUAL.csv.gz
C:/Praveen/Python/AlphaHFT/data_2024-04-01_BTC-PERPETUAL.csv.gz


In [6]:
#verify books
def validate_prices(i, curr_ts, data_df, book):
    ds = data_df.loc[curr_ts, :]
    
    if ds.loc["bid_price"] != book.loc["bids[0].price"]:
        print("bid 0 mismatch: ", i)
        print(ds.loc["bid_price"], book.loc["bids[0].price"], bdx[-1])
        
    if ds.loc["ask_price"] != book.loc["asks[0].price"]:
        print("ask 0 mismatch: ", i)
        print(ds.loc["ask_price"], book.loc["asks[0].price"], bdx[-1])

    bid_1 = ds.loc["bid_price"] * (1 + ds.loc["bid_ticks_1"] / 1000)
    if not np.allclose(bid_1, book.loc["bids[1].price"]):
        print("bid 1 mismatch: ", i)

    ask_1 = ds.loc["ask_price"] * (1 + ds.loc["ask_ticks_1"] / 1000)
    if not np.allclose(ask_1, book.loc["asks[1].price"]):
        print("ask 1 mismatch: ", i)

    bid_2 = ds.loc["bid_price"] * ds.loc["bid_ticks_2"] / 1000 + bid_1
    if not np.allclose(bid_2, book.loc["bids[2].price"]):
        print("bid 2 mismatch: ", i)

    ask_2 = ds.loc["ask_price"] * ds.loc["ask_ticks_2"] / 1000 + ask_1
    if not np.allclose(ask_2, book.loc["asks[2].price"]):
        print("ask 2 mismatch: ", i)

    bid_3 = ds.loc["bid_price"] * ds.loc["bid_ticks_3"] / 1000 + bid_2
    if not np.allclose(bid_3, book.loc["bids[3].price"]):
        print("bid 3 mismatch: ", i)

    ask_3 = ds.loc["ask_price"] * ds.loc["ask_ticks_3"] / 1000 + ask_2
    if not np.allclose(ask_3, book.loc["asks[3].price"]):
        print("ask 3 mismatch: ", i)

    bid_4 = ds.loc["bid_price"] * ds.loc["bid_ticks_4"] / 1000 + bid_3
    if not np.allclose(bid_4, book.loc["bids[4].price"]):
        print("bid 4 mismatch: ", i)

    ask_4 = ds.loc["ask_price"] * ds.loc["ask_ticks_4"] / 1000 + ask_3
    if not np.allclose(ask_4, book.loc["asks[4].price"]):
        print("ask 4 mismatch: ", i)

In [7]:
#ts = None
#book_df.index = pd.to_datetime(book_df.index, unit='us')
#book_df = book_df.resample("1s").last().ffill()

#for i in range(0, data_df.shape[0]):
#    curr_ts = data_df.index[i]
#    bdx = book_df.index[book_df.index == curr_ts]

#    book = book_df.loc[bdx[-1], :]
#    validate_prices(i, curr_ts, data_df, book)
#    break

In [8]:
data_df.shape

(86398, 46)

In [9]:
data_df.head(2)

,bid_incr,ask_incr,bid_ticks_1,bid_ticks_2,bid_ticks_3,bid_ticks_4,ask_ticks_1,ask_ticks_2,ask_ticks_3,ask_ticks_4,...,bid_vwap,ask_vwap,of1,of2,of3,of4,long_size_diff,short_size_diff,long_price_diff,short_price_diff
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-04-01 00:00:02,-0.196209,-0.350322,-0.049052,-0.077082,-0.091097,-0.007007,0.049045,0.007006,0.007006,0.056051,...,0.462750,0.407885,-8.508324e-08,-2.919157e-06,-7.147271e-08,-1.983495e-09,-0.999818,-0.859162,-71391.5,0.000000
2024-04-01 00:00:03,-0.049055,-0.189209,-0.077086,-0.042047,-0.098109,-0.049055,0.021023,0.042047,0.021023,0.028031,...,0.558403,0.124533,-5.838243e-09,-4.705398e-07,-2.806406e-06,-8.356252e-09,0.000000,-0.969572,0.0,-0.000161


In [10]:
data_df[['imbalance', 'of1']].head()

,imbalance,of1
timestamp,,
2024-04-01 00:00:02,-0.513299,-8.508324e-08
2024-04-01 00:00:03,-0.209600,-5.838243e-09
2024-04-01 00:00:04,-0.354921,-6.307055e-08
2024-04-01 00:00:05,-0.220819,1.363746e-07
2024-04-01 00:00:06,-0.931791,-6.600507e-08


In [11]:
import torch

class HftDataset(torch.utils.data.Dataset):
    def __init__(self, df, tick_size, look_back=120):
        self.data = df.drop(["bid_price", "ask_price"], axis=1)
        self.book = df[["bid_price", "ask_price"]]
        self.tick_size = tick_size
        self.look_back = look_back
      
    def __len__(self):
        return self.data.shape[0] - 11
    
    
    def __getitem__(self, index):
        X = self.data.iloc[index:index+self.look_back, :].values
        idx = self.data.index[index:index+self.look_back]
        next_idx = self.book.index[index+1:index+self.look_back+1]
        curr_bid = self.book.loc[idx, "bid_price"].values
        curr_ask = self.book.loc[idx, "ask_price"].values
        
        next_bid = self.book.loc[next_idx, "bid_price"].values
        next_ask = self.book.loc[next_idx, "ask_price"].values

        Y = np.zeros(self.look_back)

        for i in range(next_bid.shape[0]):
            y = max(next_bid[i] - curr_bid[i], 0) * max(next_ask[i] - curr_ask[i], 0) + \
                max(curr_bid[i] - next_bid[i], 0) * max(curr_ask[i] - next_ask[i], 0) 
            y = np.sqrt(y)
            Y[i] = y
        return X, Y

In [12]:
params = {'batch_size': 64,
          'shuffle': False,
          'num_workers': 0,
          'drop_last': True}

train_ds = HftDataset(data_dfs[0], 0.5)
train_loader = torch.utils.data.DataLoader(train_ds, **params)
valid_ds = HftDataset(data_dfs[1], 0.5)
val_loader = torch.utils.data.DataLoader(valid_ds, **params)
X_batch_inputs = 0
for X_batch, y_batch in train_loader:
    X_batch_inputs = X_batch.shape[2]
    print(X_batch_inputs, X_batch.shape)
    break

44 torch.Size([64, 120, 44])


In [24]:
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """The __init__ method that initiates a GRU instance.

        Args:
            input_dim (int): The number of nodes in the input layer
            hidden_dim (int): The number of nodes in each layer
            layer_dim (int): The number of layers in the network
            output_dim (int): The number of nodes in the output layer
            dropout_prob (float): The probability of nodes being dropped out

        """
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim 

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """The forward method takes input tensor x and does forward propagation

        Args:
            x (torch.Tensor): The input tensor of the shape (batch size, sequence length, input_dim)

        Returns:
            torch.Tensor: The output tensor of the shape (batch size, output_dim)

        """
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim,device=x.device).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [37]:
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

class Optimization:
    """Optimization is a helper class that allows training, validation, prediction.
    """
    def __init__(self, model, loss_fn, optimizer):
        """
        Args:
            model (RNNModel, LSTMModel, GRUModel): Model class created for the type of RNN
            loss_fn (torch.nn.modules.Loss): Loss function to calculate the losses
            optimizer (torch.optim.Optimizer): Optimizer function to optimize the loss function
        """
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        
    def train_step(self, x, y):
        """The method train_step completes one step of training.

        Given the features (x) and the target values (y) tensors, the method completes
        one step of the training. First, it activates the train mode to enable back prop.
        After generating predicted values (yhat) by doing forward propagation, it calculates
        the losses by using the loss function. Then, it computes the gradients by doing
        back propagation and updates the weights by calling step() function.

        Args:
            x (torch.Tensor): Tensor for features to train one step
            y (torch.Tensor): Tensor for target values to calculate losses

        """
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        """The method train performs the model training

        The method takes DataLoaders for training and validation datasets, batch size for
        mini-batch training, number of epochs to train, and number of features as inputs.
        Then, it carries out the training by iteratively calling the method train_step for
        n_epochs times. If early stopping is enabled, then it  checks the stopping condition
        to decide whether the training needs to halt before n_epochs steps. Finally, it saves
        the model in a designated file path.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader that stores training data
            val_loader (torch.utils.data.DataLoader): DataLoader that stores validation data
            batch_size (int): Batch size for mini-batch training
            n_epochs (int): Number of epochs, i.e., train steps, to train
            n_features (int): Number of feature columns

        """
        model_path = f'{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            print("epoch:", epoch)
            for x_batch, y_batch in tqdm(train_loader):
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch[:, -1])
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val[:, -1], yhat[:, 0]).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

        torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader, batch_size=1, n_features=1):
        """The method evaluate performs the model evaluation

        The method takes DataLoaders for the test dataset, batch size for mini-batch testing,
        and number of features as inputs. Similar to the model validation, it iteratively
        predicts the target values and calculates losses. Then, it returns two lists that
        hold the predictions and the actual values.

        Note:
            This method assumes that the prediction from the previous step is available at
            the time of the prediction, and only does one-step prediction into the future.

        Args:
            test_loader (torch.utils.data.DataLoader): DataLoader that stores test data
            batch_size (int): Batch size for mini-batch training
            n_features (int): Number of feature columns

        Returns:
            list[float]: The values predicted by the model
            list[float]: The actual values in the test set.

        """
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.to(device)
                self.model.eval()
                yhat = self.model(x_test)
                yhat=yhat.cpu().data.numpy()
                predictions.append(yhat)
                y_test=y_test.cpu().data.numpy()
                values.append(y_test)

        return predictions, values

    def plot_losses(self):
        """The method plots the calculated loss values for training and validation
        """
        plt.style.use('ggplot')
        plt.figure(figsize=(10,5))
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [38]:
import torch.optim as optim
def General_Settings(input_dim):
        output_dim = 1
        hidden_dim = 64
        layer_dim = 3
        batch_size = 64
        dropout = 0.2
        n_epochs = 20
        learning_rate = 1e-3
        weight_decay = 1e-6

        model_params = {'input_dim': input_dim,
                        'hidden_dim' : hidden_dim,
                        'layer_dim' : layer_dim,
                        'output_dim' : output_dim,
                        'dropout_prob' : dropout}
        
        model = GRUModel(**model_params)
        loss_fn = nn.MSELoss(reduction="mean")
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        opt = Optimization(model=model.to(device), loss_fn=loss_fn, optimizer=optimizer)
        opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
        opt.plot_losses()

        predictions, values = opt.evaluate(
            test_loader_one,
            batch_size=1,
            n_features=input_dim
        )
        return predictions, values

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def format_predictions(predictions, values):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={"value": vals, "prediction": preds})
    return df_result


def calculate_metrics(df):
    result_metrics = {'mae' : mean_absolute_error(df.value, df.prediction),
                      'rmse' : mean_squared_error(df.value, df.prediction) ** 0.5,
                      'r2' : r2_score(df.value, df.prediction)}
    
    print("Mean Absolute Error:       ", result_metrics["mae"])
    print("Root Mean Squared Error:   ", result_metrics["rmse"])
    print("R^2 Score:                 ", result_metrics["r2"])
    return result_metrics

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float64)
predictions, values=General_Settings(X_batch_inputs)
df_result=format_predictions(predictions, values)
print(calculate_metrics(df_result))

epoch: 1


100%|█████████████████████████████████████████████████████████████████████████████▉| 1348/1349 [23:45<00:01,  1.06s/it]


RuntimeError: stack expects each tensor to be equal size, but got [120, 44] at entry 0 and [119, 44] at entry 1